# Digital Transformation Advisory

## 01 - Document Collection - Procurement Plan

Purpose: to download the procurement plans/plan de adquisiciones

In [ ]:
#'''
# **************************************************************************************************************** #
#*****************************************  IDB - AUG Data Analytics  ******************************************** #
# **************************************************************************************************************** #
#
#-- Notebook Number: 01 - Document Collection - Procurement Plan
#-- Title: Digital Transformation Advisory
#-- Audit Segment: 
#-- Continuous Auditing: Yes
#-- System(s): Documents stored at IDBDocs, IDB SharePoint & ezShare
#-- Description: Download to a local folder all the selected documents:
#                - Procurement Plan
#                - 
#
#                                
#
#-- @author:  Emiliano Colina <emilianoco@iadb.org>
#-- Version:  0.1
#-- Last Update: 10/25/2020
#-- Last Revision Date: 10/25/2020 - Emiliano Colina <emilianoco@iadb.org> 
#                                    

# **************************************************************************************************************** #
#'''

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#### Environment Setup

In [ ]:
import os
import pandas as pd

In [ ]:
import time
import datetime

In [ ]:
import requests
import re

In [ ]:
import numpy as np

In [ ]:
# Set working directory
main_dir = "C:\\Users\\emilianoco\\Desktop\\2020"
data_dir = "/Digital_Transformation"


os.chdir(main_dir + data_dir) # working directory set
print('Working folder set to: ' + os.getcwd()) # working directory check

# **************************************************************************************************************** #

#### Functions

In [ ]:
#######       
file_extension = re.compile('\.[a-zA-Z]{3}[a-zA-Z]?$')  # regular expression corrected

In [ ]:
def url_check(url):
    '''
    Description: Checks how to connect to a sharepoint/ezshare or idbdocs repository to download a file.
    (based on the pcr_validate() function)
    @author: emilianoco
    Version:
        - v0.1 - initial version (07/07/2020)
    '''
    
    url = url.strip() # remove trailing white spaces
    
    if ('idbdocs' in url) or ('ezws' in url):
        
        # protocol and host adjustment
        if url.startswith('http:'): # replace http with https
            url = url.replace('http', 'https')
        elif url.startswith('https://ezws'): # replace the ezws host with idbdocs
            url = url.replace('https://ezws', 'https://idbdocs')

        if 'EZSHARE' in url: 
            # Connect once using the cookie_2 (from idbdocs) to get the last url (in sharepoint), and then
            # connect using the sharepoint cookie
            return('connect_twice',url)

        else: 
            # connect using the cookie_2 (from idbdocs)
            return('idbdocs_directly',url)
            #r = requests.get(df_2['Link Descarga'][index], headers = h_idbdocs, allow_redirects = True)

    else:
        if 'www.iadb.org' in url:
            if 'EZSHARE' in url:
                url = url.replace('https://www.iadb.org/Document.cfm?id=', 'https://idbdocs.iadb.org/wsdocs/getdocument.aspx?docnum=')
                return('connect_twice',url)
            
        else: 
            if 'sharepoint' in url:
                
                return('sharepoint_directly', sharepoint_adjust(url))

In [ ]:
def sharepoint_adjust(original):
    '''
    Description: Adjusts a sharepoint url to download the file it points to.
    @author: camilode; emilianoco
    
    Version: 
        - v0.3 - added control for trailing parameters in URL (07/04/2020)
        - v0.2 - Added control for url with path 'WopiFrame.aspx' 
        - v0.1 - (01/09/2020)
    '''
    #posicion_corte = 0
    #del posicion_corte
    if '{' in original:
        original = original.replace('{','%7B')
    if '}' in original:
        original = original.replace('}', '%7D')
    
    if '%7D' in original:
        posicion_corte = original.find('%7D')
        original = original[:posicion_corte]
    #print(original)

    if 'Doc.aspx' in original: 
        original = original.replace('Doc.aspx?sourcedoc=%7B','download.aspx?UniqueId=')
    
    if 'WopiFrame.aspx' in original:
        original = original.replace('WopiFrame.aspx?sourcedoc=%7B','download.aspx?UniqueId=')

    return(original)


In [ ]:
def check_content(req):
    '''
    Description: Checks for certain messages/errors in a request content
    @ author: emilianoco
    Version:
        - v0.1 - initial version - (07/07/2020)
    '''
    if 'could not be found in Sharepoint EzShare' in str(req.content):
        return('not found')
    elif ('AccessDenied.aspx' in str(req.content)) or ('does not have permissions to access this resource' in str(req.content)): 
        return('access denied')
    else:
        return('content undefined')

In [ ]:
def download_file(file_dir, req, name_prefix=''):
    '''
    Description: Saves to 'file_dir' the file under 'req' as 'file_name', obtained either from the URL or 
    the HTTP response. Optional parameter added to insert a prefix to the file_name.
    
    The function also controls 'file_dir' + 'file_name' lengths to avoid OS constraints.
    
    To control duplicates and not overwrite already downloaded files, the function iterates over
    the destination folder and adds a counter if the 'file_name' is already present.
    
    @ author: emilianoco
    
    Version:
        - v0.4 - optional parameter (07/07/2020)
        - v0.3 - (06/17/2020)
        - v0.2 - (06/16/2020)
        - v0.1 - (01/02/2020)
    '''
    
    if req.headers.get('Content-Disposition') == None: 
        #file_name not in 'Content-Disposition' but in in url - usually sharepoint
        if '&file=' in requests.utils.unquote(req.request.url.split('/')[-1], encoding='utf-8', errors='replace'):
            
            # case where the file_name is defined in parameter &file, usually a 'docx' file
            file_name_orig = requests.utils.unquote(req.request.url.split('/')[-1], encoding='utf-8', errors='replace').split('&file=')[-1].split('&')[0]
            
            # the request url needs to be re-written and a new connection is required:
            req = requests.get(sharepoint_adjust(req.url), headers = h_sharepoint, allow_redirects = True) 
            
        else:
            file_name_orig = requests.utils.unquote(req.request.url.split('/')[-1], encoding='utf-8', errors='replace')
    else:
        #file_name extracted from the content - usually idbdocs
        file_name_orig = requests.utils.unquote(req.headers['Content-Disposition'].split('filename=')[-1].encode('latin-1').decode('utf-8')).replace('"', '')
    
    
    # Set name_prefix (v0.4)
    if name_prefix != '':
        #not empty
        name_prefix = name_prefix + '_'
    
    
    # Check file_name length (v0.4)
    if len(file_dir + name_prefix + file_name_orig) > 240: 
        file_name_ini = file_name_orig[0:180]
        file_name_fin = file_name_orig[-20:]
        file_name = name_prefix + file_name_ini + '~' + file_name_fin
    else:
        file_name = name_prefix + file_name_orig
    

    # Check if file_name already present in destination folder (v0.4)
    if file_name in os.listdir(file_dir):
        file_name = ''.join(file_name.split('.')[:-1]) + '_' + '%s' + str('.') + file_name.split('.')[-1]
        i = 1
        while os.path.exists(file_dir + '\\' + file_name %i):
            i += 1
    
        # Save the file
        with open(file_dir + '\\' + file_name %i, 'wb') as f:
            f.write(req.content)
        print('Downloaded: ' + file_name %i)   #v0.3
        return file_name %i 
    
    else:
        with open(file_dir + '\\' + file_name, 'wb') as f:
            f.write(req.content)
        print('Downloaded: ' + file_name)
        return file_name

# **************************************************************************************************************** #

#### Headers configuration 

In [ ]:
### The following variables must be set! ###
cookie_idbdocs = 'XXXXX'    ## <----
cookie_sharepoint = 'YYYYY' ## <----

In [ ]:
# idbdocs
h_idbdocs = {
    'method': 'GET',
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'cookie': cookie_idbdocs,
    'accept-encoding': 'gzip, deflate, br',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36', #Chrome
    }

In [ ]:
# sharepoint
h_sharepoint = {
    'method': 'GET',
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'cookie': cookie_sharepoint,
    'accept-encoding': 'gzip, deflate, br',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36', #Chrome
    }

# **************************************************************************************************************** #

In [ ]:
## cookies & headers clean-up:
#del cookie_idbdocs
#del cookie_sharepoint
#del h_idbdocs
#del h_sharepoint

In [ ]:
# **************************************************************************************************************** #

# Procurement Plan

#### Data Preparation & Exploration

In [ ]:
# Load source xlsx file:
data_ = pd.read_excel(r"./input/Data-30 Sep 2020-All documents.xlsx", sheet_name='procurement_plan')

In [ ]:
data_.head()

In [ ]:
data_.shape

In [ ]:
#print(data_.columns)

In [ ]:
print(data_.columns)
data = data_[['OPERATION_ID', 'operation_number', 'Oper_type', 'Country', 'Region', 'Sector', 'Sector_Subsector', 'OPERATION_NAME_ES', \
              'OPERATION_YEAR', 'APPROVAL_DATE', 'DOCUMENT_ID', 'DOCUMENT_REFERENCE', 'DESCRIPTION']].copy()

In [ ]:
# **************************************************************************************************************** #

In [ ]:
# **************************************************************************************************************** #

### Procurement Documents Collection

In [ ]:
# We will work with the filtered dataframe, and we'll add additional columns for storing the results:
df = data.copy()

In [ ]:
# for storing the document's name and its status, i.e.: 'downloaded', exception message 
df['Document_Name'] = '' #
df['Document_Status'] = '' #

In [ ]:
df['Document_URL'] = ''

In [ ]:
df.head()

There are operations with 'NULL' value in the URL.
<br>
Using the EzShare code, fill in the `URL_2` field by including the common idbdocs url to request as:
URL base: `"https://idbdocs.iadb.org/wsdocs/getDocument.aspx?DOCNUM="` + `"Codigo_EZSHARE"`

In [ ]:
url_base = 'https://idbdocs.iadb.org/wsdocs/getDocument.aspx?DOCNUM='

# new column to store idbdocs_url + ezshare_code:
df['URL_2'] = ''

for index, row in df.iterrows():
    print('Processing index:', str(index))
    url = url_base + df['DOCUMENT_REFERENCE'][index].strip()
    df.at[index, 'URL_2'] = url

In [ ]:
# Destination folder setup: all files will be downloaded to 'file_dir'

desktop_dir = "C:\\Users\\emilianoco\\Desktop"
file_dir = desktop_dir + "\\procur_plans"

print(file_dir)

In [ ]:
df.rename(columns={'operation_number':'OPERATION_NUMBER'}, inplace=True)

In [ ]:
df.head()

### All Documents collection:

In [ ]:
%%time

t = 1     # counter set

for index, row in df.iterrows():
    print('## processing index', str(index))
    
    checked_message, checked_url = url_check(df['URL_2'][index])
    print(checked_message, checked_url)
    r = requests.get(checked_url, headers = h_idbdocs, allow_redirects = True)

    if 'application/' in r.headers['Content-Type']:
        # download
        print('download document')
        file_name = download_file(file_dir, r, df['OPERATION_NUMBER'][index]) 
        df.at[index, 'Document_Name'] = file_name
        df.at[index, 'Document_Status'] = 'OK - direct download'
        df.at[index, 'Document_URL'] = r.url

    else: 
        status = check_content(r)
        if status in ['access denied','not found']:
            print(status)
            print('save result and break')
            df.at[index, 'Document_Name'] = 'not downloaded'
            df.at[index, 'Document_Status'] = status
        
        else:
            print('continue')
            print('... checking request.history[i].url ...')
            for i in range(len(r.history)):      # cross-site authentication control
                if bool(re.search(r'\.[a-z]{3}[a-z]?(\?d\=[a-z0-9]+)?$',r.history[i].request.url.lower())) and not ('Authenticate.aspx' in str(r.history[i].request.url)): # added control when url ends in ?d=...
                    location = r.history[i].url  # effective URL after the redirects
                    print(location)
                    
                    if 'sharepoint' in location:  # connect using sharepoint headers and cookie
                        s = requests.get(location, headers = h_sharepoint, allow_redirects = True)
                        
                        status = check_content(s)
                        if status in ['access denied','not found']:
                            print('***')
                            print(status)
                            print('save result and break')
                            df.at[index, 'Document_Name'] = 'not downloaded'
                            df.at[index, 'Document_Status'] = status
                            print('***')
                            
                        else: 
                            print('try downloading from', location)
                            try:
                                file_name = download_file(file_dir, s, df['OPERATION_NUMBER'][index])             # download the document and get the filename
                                df.at[index, 'Document_Name'] = file_name
                                df.at[index, 'Document_Status'] = 'OK - download from redirect'
                                df.at[index, 'Document_URL'] = s.url
                                print('Downloaded!')
                                #count =+ 1 
                                break
                                
                            except Exception as e: 
                                df.at[index, 'Document_Name'] = 'Not downloaded'
                                df.at[index, 'Document_Status'] = str(e)
                                print('Not downloaded: '+ str(e))
                            
                    
                    else: 
                        print('\'sharepoint\' not found in url', location)
     
                    del location
    print('##')
    print('')
    t = t + 1
    if (t % 7) == 0:
        print('')
        print("* * 2 seconds pause")
        time.sleep(2) # 2 sec pause inserted every 7 docs
        print('')
    print('')

In [ ]:
#pending_1 = df[df.Document_Status.str.contains('Errno 2')].index

In [ ]:
df[df.Document_Status == 'not found']

In [ ]:
df['URL_2'][12]

In [ ]:
df[df.duplicated(subset=['OPERATION_NUMBER'])].shape

In [ ]:
df[df.sort_values(['OPERATION_NUMBER', 'DOCUMENT_ID']).duplicated(subset=['OPERATION_NUMBER'], keep='last')]

In [ ]:
df_test = df.sort_values(['OPERATION_NUMBER', 'DOCUMENT_ID'])

In [ ]:
#save results to excel:
df_test[~df_test.duplicated(subset=['OPERATION_NUMBER'], keep='last')].to_excel('./output/Procurement_Plan_doc_collection_filtered.xlsx')

In [ ]:
import joblib
# save completed df:
# Store results: 
# v0.1 - 10/25: 
joblib.dump(df, './output/' + 'Procurment_Plan_Doc_Collection_2020-10-25_all.joblib' + '.bz2', compress=('bz2', 3))

In [ ]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None

    (c) [MaxU](https://stackoverflow.com/users/5741205/maxu?tab=profile)
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)
        
        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)
        
        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()
            

In [ ]:
append_df_to_excel('./input/Data-30 Sep 2020-All documents - original.xlsx', df, sheet_name='documents_status', index=True)

In [ ]:
append_df_to_excel('./input/Data-30 Sep 2020-All documents - original.xlsx', df_test[~df_test.duplicated(subset=['OPERATION_NUMBER'], keep='last')], sheet_name='duplicates_filtered', index=True)

In [ ]:
#output_file_name = './input/Data-30 Sep 2020-All documents.xlsx' # file name
### Output to new Excel containing each test on a different sheet

#with pd.ExcelWriter(output_file_name) as writer:
#    df.to_excel(writer, sheet_name='documents_status')
#    df_test[~df_test.duplicated(subset=['OPERATION_NUMBER'], keep='last')].to_excel(writer, sheet_name='duplicates_filtered')

In [ ]:
df_test[~df_test.duplicated(subset=['OPERATION_NUMBER'], keep='last')][df_test.Document_Status != 'not found']

In [ ]:
# **************************************************************************************************************** #

# ************** #

In [ ]:
# FIN

In [ ]:
#'''
# ******************************************************************************************************************** #
# *************************************************  Version Control  ************************************************ #
# ******************************************************************************************************************** #
  
#   Version:            Date:                User:                    Change:                                          #                                                          

#   - 0.1           10/25/2020        Emiliano Colina       - Initial version, including filtering duplicates
#                                                            

#
# ******************************************************************************************************************** #
#'''
